In [1]:
import os
import pandas as pd

### Load data ###

In [2]:
cpd = {}

for dataset in ["CPU", "Memory", "Disk"]:

    results_dir = "./%s_results" % dataset
    if dataset not in cpd.keys():
        cpd[dataset] = {} 

    for f in os.listdir("%s" % results_dir):
        # Exclude hidden files (most likely thumbnails)
        if f[:2] != "._":
            print("Loading: %s/%s" % (results_dir, f))
            df = pd.read_csv("%s/%s" % (results_dir, f))
            print(len(df))

            if "aggr" in f:
                # Extract CPD threshold from the filename 
                lthreshold = float(f.split("-")[1])

                df["lthreshold"] = lthreshold
                if "aggr" not in cpd[dataset].keys():
                    cpd[dataset]["aggr"] = pd.DataFrame()
                cpd[dataset]["aggr"] = pd.concat([cpd[dataset]["aggr"], df], sort=False)

            if "indiv" in f:
                if "indiv" not in cpd[dataset].keys():
                    cpd[dataset]["indiv"] = pd.DataFrame()
                cpd[dataset]["indiv"] = pd.concat([cpd[dataset]["indiv"], df], sort=False)

            if "meta" in f:
                cpd[dataset]["meta"] = df

            if "segment-durations" in f:
                cpd[dataset]["segment-durations"] = df

Loading: ./CPU_results/indiv-0.70
283
Loading: ./CPU_results/aggr-1.00
4034
Loading: ./CPU_results/indiv-0.40
147
Loading: ./CPU_results/aggr-0.30
354
Loading: ./CPU_results/aggr-0.90
3951
Loading: ./CPU_results/aggr-0.70
3068
Loading: ./CPU_results/indiv-1.00
410
Loading: ./CPU_results/aggr-0.40
930
Loading: ./CPU_results/meta
11
Loading: ./CPU_results/indiv-0.30
49
Loading: ./CPU_results/indiv-0.90
363
Loading: ./CPU_results/indiv-0.80
325
Loading: ./CPU_results/aggr-0.50
1944
Loading: ./CPU_results/segment-durations
6725
Loading: ./CPU_results/aggr-0.60
2459
Loading: ./CPU_results/aggr-0.80
3565
Loading: ./CPU_results/indiv-0.50
193
Loading: ./CPU_results/indiv-0.60
235
Loading: ./Memory_results/indiv-0.70
1406
Loading: ./Memory_results/aggr-1.00
6241
Loading: ./Memory_results/indiv-0.40
925
Loading: ./Memory_results/aggr-0.30
2656
Loading: ./Memory_results/aggr-0.90
6002
Loading: ./Memory_results/aggr-0.70
5420
Loading: ./Memory_results/indiv-1.00
1784
Loading: ./Memory_results/agg

In [3]:
count_df = pd.DataFrame(columns=["CPU: CP #", "Mem: CP #", "Disk: CP #"])
for lthreshold in cpd["CPU"]["indiv"]["lthreshold"].sort_values().unique():
    cpu_cpd_c = len(cpd["CPU"]["indiv"][(cpd["CPU"]["indiv"]["lthreshold"] == lthreshold)])
    
    mem_cpd_c = len(cpd["Memory"]["indiv"][(cpd["Memory"]["indiv"]["lthreshold"] == lthreshold)])
    
    disk_cpd_c = len(cpd["Disk"]["indiv"][(cpd["Disk"]["indiv"]["lthreshold"] == lthreshold)])
    
    count_df.loc[lthreshold] = (cpu_cpd_c, mem_cpd_c, disk_cpd_c)
    
count_df["Total #"] = count_df.sum(1)
count_df.loc['Total',:] = count_df.sum(0)
display(count_df)

,CPU: CP #,Mem: CP #,Disk: CP #,Total #
0.3,49,492,42,583.0
0.4,147,925,76,1148.0
0.5,193,1113,108,1414.0
0.6,235,1263,134,1632.0
0.7,283,1406,160,1849.0
0.8,325,1529,191,2045.0
0.9,363,1630,215,2208.0
1.0,410,1784,245,2439.0
Total,2005,10142,1171,13318.0


In [4]:
cpd['Memory']['indiv']

,hw_type,lthreshold,testname,dvfs,socket_num,timestamp,percent_change
0,c220g1,0.7,add,no,0,1515202534,-3.772623
1,c220g1,0.7,add,no,0,1540060856,9.972447
2,c220g1,0.7,add,no,0,1543671468,-10.811279
3,c220g1,0.7,add,no,1,1515202534,-3.804811
4,c220g1,0.7,add,no,1,1540060985,-1.383751
...,...,...,...,...,...,...,...
1258,xl170,0.6,write_memory_rep_stosq_omp,yes,0,1572632379,5.007408
1259,xl170,0.6,write_memory_sse,no,0,1572632379,6.229457
1260,xl170,0.6,write_memory_sse,yes,0,1572632379,6.192962
1261,xl170,0.6,write_memory_sse_omp,no,0,1572632379,6.462294


In [7]:
cpd['Memory']['segment-durations']

,hw_type,lthreshold,testname,dvfs,socket_num,segment_duration
0,c220g1,0.3,add,no,0,493.256701
1,c220g1,0.3,add,no,0,487.792813
2,c220g1,0.3,add,no,1,490.258171
3,c220g1,0.3,add,no,1,490.791343
4,c220g1,0.3,add,yes,0,1009.429282
...,...,...,...,...,...,...
18441,xl170,1.0,write_memory_sse_omp,no,0,442.500336
18442,xl170,1.0,write_memory_sse_omp,no,0,114.541678
18443,xl170,1.0,write_memory_sse_omp,yes,0,234.249259
18444,xl170,1.0,write_memory_sse_omp,yes,0,114.041863


In [32]:
cpd['Memory']['segment-durations'][cpd['Memory']['segment-durations']['lthreshold']==0.3]

,hw_type,lthreshold,testname,dvfs,socket_num,segment_duration
0,c220g1,0.3,add,no,0,493.256701
1,c220g1,0.3,add,no,0,487.792813
2,c220g1,0.3,add,no,1,490.258171
3,c220g1,0.3,add,no,1,490.791343
4,c220g1,0.3,add,yes,0,1009.429282
...,...,...,...,...,...,...
17382,xl170,0.3,write_memory_rep_stosq_omp,yes,0,557.042014
17383,xl170,0.3,write_memory_sse,no,0,557.042014
17384,xl170,0.3,write_memory_sse,yes,0,557.042014
17385,xl170,0.3,write_memory_sse_omp,no,0,557.042014


In [24]:
cpd['Memory']['meta']

,hw_type,total_measurements,configurations,start_timestamp,end_timestamp
0,c220g1,372416,128,1495272056,1582486746
1,c220g5,672512,128,1534358035,1582486667
2,c6220,174208,128,1534358368,1582487158
3,c6320,443648,128,1495272018,1582487946
4,c6420,104832,64,1534359008,1582466277
5,c8220,649088,128,1495457110,1582487883
6,d430,695296,128,1534357194,1582485700
7,m400,200970,14,1495271912,1582442353
8,m510,573088,64,1495476412,1582485523
9,r320,114112,64,1534358385,1581234375


In [20]:
memory = pd.merge(cpd['Memory']['indiv'], cpd['Memory']['segment-durations'], on=['hw_type','lthreshold','testname','dvfs','socket_num'])

In [28]:
memory

,hw_type,lthreshold,testname,dvfs,socket_num,timestamp,percent_change,segment_duration
0,c220g1,0.7,add,no,0,1515202534,-3.772623,202.297060
1,c220g1,0.7,add,no,0,1515202534,-3.772623,449.250903
2,c220g1,0.7,add,no,0,1515202534,-3.772623,287.712060
3,c220g1,0.7,add,no,0,1515202534,-3.772623,41.789491
4,c220g1,0.7,add,no,0,1540060856,9.972447,202.297060
...,...,...,...,...,...,...,...,...
35583,xl170,0.6,write_memory_sse,yes,0,1572632379,6.192962,114.041863
35584,xl170,0.6,write_memory_sse_omp,no,0,1572632379,6.462294,443.000150
35585,xl170,0.6,write_memory_sse_omp,no,0,1572632379,6.462294,114.041863
35586,xl170,0.6,write_memory_sse_omp,yes,0,1572632379,6.550455,443.000150


In [30]:
memory[memory['lthreshold']==0.3]

,hw_type,lthreshold,testname,dvfs,socket_num,timestamp,percent_change,segment_duration
14868,c220g1,0.3,add,no,0,1540341447,-1.950813,493.256701
14869,c220g1,0.3,add,no,0,1540341447,-1.950813,487.792813
14870,c220g1,0.3,add,no,1,1540082374,-1.674475,490.258171
14871,c220g1,0.3,add,no,1,1540082374,-1.674475,490.791343
14872,c220g1,0.3,copy,no,0,1542289064,-0.658396,515.798565
...,...,...,...,...,...,...,...,...
16127,m510,0.3,write_memory_sse_omp,yes,0,1534183935,-2.778777,307.791794
16128,m510,0.3,write_memory_sse_omp,yes,0,1534183935,-2.778777,251.254363
16129,m510,0.3,write_memory_sse_omp,yes,0,1555892312,0.833686,448.003738
16130,m510,0.3,write_memory_sse_omp,yes,0,1555892312,0.833686,307.791794
